In [1]:
import json
import asyncio
import random
from zhipuai import ZhipuAI
import pandas as pd
from rouge import Rouge
import nltk
import nest_asyncio
import time
nest_asyncio.apply()
dataset_list = []

with open("development.json", "r") as file:
    seed_data = json.load(file)

with open("shopping_chatglm_data.json", "r") as file:
    dataset_list = json.load(file)
data_str = [item["text"] for item in dataset_list]
df_seed = pd.DataFrame(seed_data)

In [2]:
def split_string(gpt_output: str):
    parts = gpt_output.split('###')
    return parts

def delete_repeat(threshold, dataset_list: list, tar):
    if len(dataset_list) == 0:
        return True
    for item in dataset_list:
        rouge = Rouge()
        scores = rouge.get_scores([item], [tar])
        if scores[0]['rouge-l']['f'] > threshold:
            return False
    return True

async def create_request(client, model, messages):
    response = client.chat.asyncCompletions.create(model=model, messages=messages)
    return response.id

flag = 0
async def check_task_status(client, task_id):
    get_cnt = 0
    while True:
        result_response = client.chat.asyncCompletions.retrieve_completion_result(id=task_id)
        # print(result_response)
        task_status = result_response.task_status
        if task_status == 'FAILED':
            return [task_status, None]
        elif task_status == 'SUCCESS':
            print(str(task_id) + ' SUCCESS')
            return [task_status, result_response.dict()['choices'][0]['message']['content']]
        get_cnt += 1
    return ['FAILED', None]  # 如果超时，返回FAILED

In [4]:
client = ZhipuAI(api_key="###")
flag = 0
final_seed_len = 5
generate_time = 1
num_list = list(range(len(df_seed)))
selected_set = []
prefix_prompt = """
You are asked to come up with a set of 2 diverse tasks. These task instructions will be given to a GPT model and we will evaluate the GPT model for completing the instructions.

Here are the requirements:
0. The theme of this task is about Shopping Concept Understanding, aims to evaluate the model's ability to understand shopping entities in the form of texts, such as product names, product categories, attributes, product descriptions, reviews, etc
1. Try not to repeat the verb for each instruction to maximize diversity.
2. The way of expressing words in the input_field needs to be diversified.
3. In order to make the type of task be diverse, trying to generate task types different from the examples. The list should include diverse types of tasks like open-ended generation, classification, ranking, multi-choice etc.
4. A GPT language model should be able to complete the output_field based on the input_field. 
5. The output_field should be an appropriate and correct response to the input_field. Make sure the output_field is less than 100 words.
6. Each task needs to be enclosed with “###” just like the examples. Note that the task type should be at the beginning of the example.
7. input_field must end with 'Answer ' or 'Output: '. 
List of 5 tasks:
"""

In [5]:
prompt_gpt_input_list = []

for i in range(generate_time):
     while True:
        selected_numbers = set(random.sample(num_list, final_seed_len))
        if selected_numbers not in selected_set:
            selected_set.append(selected_numbers)
            break

for set_item in selected_set[-generate_time:]:
    df_final_seed = pd.DataFrame([], columns = ["input_field", "output_field", "task_name", "task_type", "metric", "is_multiple_choice"])
    item_list = ""
    for j in set_item:
        df_final_seed.loc[len(df_final_seed)] = df_seed.loc[j]
    for j in range(len(df_final_seed)):
        temp_string = '\n###\n task_type:' + df_final_seed['task_type'][j] + "\n input_field: " + str(df_final_seed['input_field'][j]) + '\n output_field: ' + str(df_final_seed['output_field'][j]) + '\n'
        item_list += temp_string
    prompt_gpt_input = prefix_prompt + item_list
    prompt_gpt_input_list.append(prompt_gpt_input)


async def main():
    task_ids = []
    for input in prompt_gpt_input_list:
        task_id = await create_request(client, "glm-4", [{'role': 'user', 'content': input}])
        task_ids.append(task_id)

    for task_id in task_ids:
        task_status = await check_task_status(client, task_id)
        #print(f"任务 {task_id} 的状态是： {task_status}")
        if task_status[0] == 'SUCCESS':
            tar_str_list = split_string(task_status[1])
            for tar in tar_str_list:
                if tar != "" and delete_repeat(0.7, data_str, tar):
                    dataset_list.append({"text": tar})
                    data_str.append(tar)

asyncio.run(main())

APIInternalError: Error code: 500, with error text {"error":{"code":"106","message":"AMOUNT_NOT_ENOUGH"}}

In [13]:
json_data = json.dumps(dataset_list, indent=4, separators=(',', ': '))
filename = 'shopping_chatglm_data.json'
# 将JSON字符串写入文件
with open(filename, 'w') as f:
    f.write(json_data)

In [14]:
prompt_gpt_input_list

["\nYou are asked to come up with a set of 20 diverse task. These task instructions will be given to a GPT model and we will evaluate the GPT model for completing the instructions.\n\nHere are the requirements:\n0. The theme of this task is about Shopping Concept Understanding, aims to evaluate the model's ability to understand shopping entities in the form of texts, such as product names, product categories, attributes, product descriptions, reviews, etc\n1. Try not to repeat the verb for each instruction to maximize diversity.\n2. The way of expressing words in the input_field needs to be diversified.\n3. In order to make the type of task be diverse, trying to generate task types different from the examples. The list should include diverse types of tasks like open-ended generation, classification, ranking, multi-choice etc.\n4. A GPT language model should be able to complete the output_field based on the input_field. \n5. The output_field should be an appropriate and correct response